In [1]:
# %load imdb_data_to_graph.py
import numpy as np
import imbd_data_loader
import dgl
import pickle
import os
from sklearn.preprocessing import MultiLabelBinarizer
import torch

def embed_genre():
    return
def embed_movie_info(movie_id,id2str_info_dic,id2numer_info_dic,id2genre_dic,nbr_movie_feat,mov2id_dict,uniq_mov_types):
    feat = torch.zeros((nbr_movie_feat))
    if movie_id in id2str_info_dic:
        feat[0:600]=torch.tensor(id2str_info_dic[movie_id])
    if movie_id in id2numer_info_dic:
        if len(id2numer_info_dic[movie_id])==7:
            row=id2numer_info_dic[movie_id]
            if row[1] is None:
                row[1]=0
            if row[4] is None:
                row[4]=0
            feat[600:606]=torch.cat((torch.tensor(row[0:2]),torch.tensor(row[3:])),0)
            feat[606:606+uniq_mov_types]=torch.tensor(mov2id_dict[row[2]])
    if movie_id in id2genre_dic:
          feat[606+uniq_mov_types:635+uniq_mov_types]=torch.tensor(id2genre_dic[movie_id].todense())
    #print(feat)
    
    return feat
def embed_person_info(person_id,people_id2name_dic,people_id2primaryProfession,feat_nbr):
    feat = torch.zeros((feat_nbr))
    if person_id in people_id2name_dic:
        row=people_id2name_dic[person_id][1:3]
        if row[0] is None:
            row[0]=0
        if row[1] is None:
            row[1]=0
        feat[0:2]=torch.tensor(row)
    if person_id in people_id2primaryProfession:
        feat[2:]=torch.tensor(people_id2primaryProfession[person_id].todense())
    return feat

print('hi')

hi


In [2]:
IMDB_DIR='../data/imdb_data/'
id2numer_info_dic, id2str_info_dic, id2genre_dic, id2l_director_dic, id2l_writer_dic, \
id2l_principal_dic, people_id2name_dic, people_id2primaryProfession= imbd_data_loader.load_imdb_subset_dics()

IMDb dics loaded ...


In [3]:
    use_principals = True

    # Generate edge lists
    id2l_director_edge_list = [(k, d) for k, v in id2l_director_dic.items() for d in v]
    id2l_writer_edge_list = [(k, w) for k, v in id2l_writer_dic.items() for w in v]
    id2l_principal_edge_list = [(k, w) for k, v in id2l_principal_dic.items() for w in v]
    id2l_peopleknownformovies_edge_list =[(k, d) for k, v in people_id2name_dic.items() for d in v[3].split(',')]
    # extract ids that participate in the abore relationships
    # extract ids of movies
    moviesdirected = [i[0] for i in id2l_director_edge_list]
    movieswithprincipals=[i[0] for i in id2l_principal_edge_list]
    movieswritten = [i[0] for i in id2l_writer_edge_list]
    moviesknwonforperson =[i[1] for i in id2l_peopleknownformovies_edge_list ]
    
    # edge feature information among principal and movie
    dictOfPrincipals = {(i[0],i[1][0]):i[1][1:] for i in id2l_principal_edge_list}

    # extract ids of persons
    directors = [i[1] for i in id2l_director_edge_list]
    principals = [i[1][0] for i in id2l_principal_edge_list]
    writers = [i[1] for i in id2l_writer_edge_list]
    personsknownformovies=[i[0] for i in id2l_peopleknownformovies_edge_list ]
    #print(id2l_director_edge_list)

    # Need to map all ids to int
    # create list of all movies and people
    if use_principals:
        movie_imdb = np.concatenate((moviesdirected, movieswritten,movieswithprincipals,moviesknwonforperson))
        people_imdb = np.concatenate(( directors, writers,principals,personsknownformovies))
    else:
        movie_imdb = np.concatenate((moviesdirected, movieswritten))
        people_imdb = np.concatenate((directors, writers))
    # extract unique ids movies and people
    uniq_mov, edges_mov = np.unique(movie_imdb, return_inverse=True)
    uniq_people, edges_peopl = np.unique(people_imdb, return_inverse=True)

    # map the uniq ids to integers
    dictOfMovies = {uniq_mov[i]: i for i in range(len(uniq_mov))}
    dictOfPeople = {uniq_people[i]: i for i in range(len(uniq_people))}

In [4]:
    # map string edge list to int id edge list
    intid2l_writer_edge_list = [(dictOfMovies[k], dictOfPeople[v]) for (k, v) in id2l_writer_edge_list]
    intid2l_director_edge_list = [(dictOfMovies[k], dictOfPeople[v]) for (k, v) in id2l_director_edge_list]

In [5]:
    # create graph using edge lists
    if use_principals:
        intid2_peopleknownformovies_edge_list= [(dictOfPeople[k],dictOfMovies[v])for (k, v) in id2l_peopleknownformovies_edge_list]
        intid2l_principal_edge_list = [(dictOfMovies[k], dictOfPeople[v[0]]) for (k, v) in id2l_principal_edge_list]
        g= dgl.heterograph({('movie','directed_by','person'):intid2l_director_edge_list,
                            ('movie','written_by','person'):intid2l_writer_edge_list,
                        ('movie','participated_by','person'):intid2l_principal_edge_list,
                           ('person','known_for','movie'):intid2_peopleknownformovies_edge_list})
    else:
        g = dgl.heterograph({('movie', 'directed_by', 'person'): intid2l_director_edge_list,
                             ('movie', 'written_by', 'person'): intid2l_writer_edge_list})

In [6]:
    movie_type=[v[2] if len(v)==7 else 'short' for k, v in id2numer_info_dic.items() ]
    uniq=np.unique(movie_type)
    uniq_mov_types=len(uniq)
    mov2id_dict={}
    for i in range(uniq_mov_types):
        my_one_hot=np.zeros((uniq_mov_types))
        my_one_hot[i]=1
        mov2id_dict[uniq[i]]=    my_one_hot
    
   
    

In [7]:
    # create features
    nbr_person_feat=45
    nbr_movie_feat=636+uniq_mov_types
    # TODO make the datatype float 32 smaller size
    people_feat = torch.tensor(np.zeros((len(uniq_people), nbr_person_feat)))
    movie_feat = torch.tensor(np.zeros((len(uniq_mov), nbr_movie_feat)))

In [8]:
    for i in range(len(uniq_mov)):
        movie_id=uniq_mov[i]
        movie_feat[i,:]=embed_movie_info(movie_id,id2str_info_dic,id2numer_info_dic,
                                         id2genre_dic,nbr_movie_feat,mov2id_dict,uniq_mov_types)
    print(movie_feat)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0716, -0.0487, -0.3243,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.5559,  0.4220,  0.2079,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1557,  0.1833,  0.0606,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1256,  0.1708, -0.1308,  ...,  0.0000,  0.0000,  0.0000]],
       dtype=torch.float64)


In [9]:
    for i in range(len(uniq_people)):
        person_id=uniq_people[i]
        people_feat[i,:]=embed_person_info(person_id,people_id2name_dic,people_id2primaryProfession,nbr_person_feat)

    sum(sum(people_feat))
    

tensor(1.3284e+09, dtype=torch.float64)

In [10]:
    # TODO conver to torch th.array()
    g.nodes['movie'].data['features'] = movie_feat
    # TODO check datatype
    g.nodes['person'].data['features'] = people_feat
    pickle.dump(g, open(os.path.join(IMDB_DIR, "graph_large.pickle"), "wb"),
                protocol=4);

In [11]:
 g['directed_by'].number_of_edges()

5057995

In [14]:
 g['written_by'].number_of_edges()

7761448

In [15]:
g['participated_by'].number_of_edges()

38063872

In [16]:
g['known_for'].number_of_edges()

17304989

In [17]:
 g.number_of_nodes('movie')

5982830

In [18]:
 g.number_of_nodes('person')

9914002

In [22]:
key=list(id2numer_info_dic.keys())

id2numer_info_dic[key[602]]

[1907.0, None, 'short', 0.0, None]

In [27]:
g_no_adult=g
g_no_adult.nodes['movie'].data['features']=torch.cat((g.nodes['movie'].data['features'][:,:602],g.nodes['movie'].data['features'][:,603:]),1)
adult_labels=g.nodes['movie'].data['features'][:,602]